In [1]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import numpy as np

In [2]:
data = {
    "place": ["Place1", "Place2", "Place3", "Place4"],
    "type": ["restaurant", "park", "museum", "restaurant"],
    "cuisine": ["Italian", "", "", "Chinese"],
    "location": ["downtown", "suburbs", "downtown", "midtown"],
    "price_range": ["moderate", "free", "expensive", "cheap"],
    "rating": [4.5, 4.7, 4.2, 4.0]
}
df = pd.DataFrame(data)

In [3]:
user_profile = {
    "type": "restaurant",
    "cuisine": "Italian",
    "location": "downtown",
    "price_range": "moderate"
}

In [ ]:
le_type = LabelEncoder()
le_cuisine = LabelEncoder()
le_location = LabelEncoder()
le_price_range = LabelEncoder()
df['type'] = le_type.fit_transform(df['type'])
df['cuisine'] = le_cuisine.fit_transform(df['cuisine'])
df['location'] = le_location.fit_transform(df['location'])
df['price_range'] = le_price_range.fit_transform(df['price_range'])